In [ ]:
# install virgin environment
!odsc conda install --uri <oci://my-bucket@my-namespace/path_to_tar_file>

# modify environment
!pip install ...

# publish in project-specific bucket final env with project-specific name
!odsc conda init -b <proj-specific-bucket-name> -n <your-tenancy-namespace> -a resource_principal
!odsc conda publish -s <slug> --force

Here we have to define the infrastructure operations for the first deploy and maybe the update of the infrastructure if a re-deploy is needed

* creation of log group and custom log to be used in data science pipeline
* creation of data science pipeline --> here we should in future define a wrapper to let data scientist define if to use 1 or more steps
* creation of project - task - application - publishing task to application - scheduler - scheduled task

in future maybe also add the monitoring of infrastructure

In [ ]:
import oci
from oci.logging.models import CreateLogDetails, UpdateLogDetails, CreateLogGroupDetails

# Initialize the signer with Resource Principal
resource_principal_signer = oci.auth.signers.get_resource_principals_signer()

### LOGGING STEP

In [ ]:
# CREATE LOG GROUP
# Create a logging client with the signer
log_client = oci.logging.LoggingManagementClient({}, signer=resource_principal_signer)

log_group = log_client.create_log_group(
    CreateLogGroupDetails(
        compartment_id="ocid1.compartment.oc1..aaaaaaaagds646vklqyxsj2t2lmaexhpraeak44qmyyg2rymurl45svpwcla",
        display_name="test-iaas-log-group",
        defined_tags={"Sisal":{"environment":"dev","factory":"Digital","country":"Italy"}}
    )
)


# RETRIEVE OCID FOR THE CREATED LOG GROUP
list_log_groups_response = log_client.list_log_groups( 
    compartment_id="ocid1.compartment.oc1..aaaaaaaagds646vklqyxsj2t2lmaexhpraeak44qmyyg2rymurl45svpwcla",
    sort_by="displayName",
    sort_order="DESC")

for log_group in list_log_groups_response.data:    
    if log_group.display_name == "test-iaas-log-group":
        print(f"log group ocid: {log_group.id}")
        log_group_id = log_group.id


# CREATE CUSTOM LOG
log = log_client.create_log(
    log_group_id=log_group_id,
    create_log_details=CreateLogDetails(
        display_name="test-iaas-custom-log",
        log_type="CUSTOM",
        retention_duration=30,
        defined_tags={"Sisal":{"environment":"dev","factory":"Digital","country":"Italy"}}
    )
)


# RETRIEVE CUSTOM LOG OCID
list_log_groups_response = log_client.list_logs( 
    log_group_id=log_group_id,
    sort_by="displayName",
    sort_order="DESC")

for log_group in list_log_groups_response.data:    
    if log_group.display_name == "test-iaas-custom-log":
        print(f"log ocid: {log_group.id}")
        log_group_id = log_group.id

### PIPELINE DEFINITION

### DATA INTEGRATION 

In [ ]:
# CREATE PROJECT
# Initialize the OCI Data Integration client with the signer
data_integration = oci.data_integration.DataIntegrationClient({}, signer=resource_principal_signer)

# Step 1: Create a Data Integration project
project = data_integration.create_project(
    workspace_id="ocid1.disworkspace.oc1.eu-frankfurt-1.antheljr2ow634yalr47c3ncgnbmekjdjmooy4q7nplc66byelvsas6kpb2q",
    create_project_details=oci.data_integration.models.CreateProjectDetails(
        name="test-iaas-project"
    )
)

# CREATE REST TASK
# Step 2: Create a REST task inside the project
pipeline_id = "YOUR_PIPELINE_ID"  # Replace with the actual pipeline ID
defined_tags = {"key1": "value1", "key2": "value2"}

rest_task = data_integration.create_task(
    create_task_details=oci.data_integration.models.CreateTaskDetails(
        name="test-rest-task",
        model_type="REST",
        details=oci.data_integration.models.CreateDataIntegrationTaskFromRestTask(
            endpoint="https://datascience.eu-frankfurt-1.oci.oraclecloud.com/20190101/pipelineRuns",
            method=oci.data_integration.models.CreateRestTaskDetails.MethodType.POST,
            request_headers={"Content-Type": "application/json"},
            request_body=oci.data_integration.models.CreateRestTaskFromDataLoaderTaskRestModel(
                projectId=project.id,
                compartmentId=compartment_id,
                pipelineId=pipeline_id,
                displayName=display_name,
                definedTags=defined_tags
            )
        )
    ),
    project.id
)

# VALIDATE REST TASK


# CREATE APPLICATION


# PUBLISH REST TASK IN APPLICATION


# CREATE SCHEDULER


# CREATE SCHEDULED TASK
